In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, models

# RNN (`TypeNet`)

In [ ]:
def create_model_1(feature_dim, 
                   output_dim, 
                   model_name='KDS_TypeNet-GRU-64_None_Dense-60-30-2'):
    inputA = keras.layers.Input(shape=[None, feature_dim], name='InputA')
    inputB = keras.layers.Input(shape=[feature_dim-output_dim], name='InputB')
    
    batch_1 = keras.layers.BatchNormalization()(inputA)
    gru_1 = keras.layers.GRU(64, return_sequences=True)(batch_1)
    dropout_1 = keras.layers.Dropout(0.5)(gru_1)
    batch_2 = keras.layers.BatchNormalization()(dropout_1)
    gru_out = keras.layers.GRU(64)(batch_2)
    
    concat = keras.layers.concatenate([gru_out, inputB])
    dense_1 = keras.layers.Dense(60)(concat)
    concat_output = keras.layers.Dense(30)(dense_1)
    output = keras.layers.Dense(output_dim)(concat_output)
    model = keras.Model(inputs=[inputA, inputB], outputs=[output], name=model_name)
    return model

In [ ]:
def create_model_2(feature_dim, 
                   output_dim, 
                   model_name='KDS_TypeNet-GRU-64-dropout_None_Dense-60-30-2-Dropout'):
    inputA = keras.layers.Input(shape=[None, feature_dim], name='InputA')
    inputB = keras.layers.Input(shape=[feature_dim-output_dim], name='InputB')
    
    batch_1 = keras.layers.BatchNormalization()(inputA)
    gru_1 = keras.layers.GRU(64, return_sequences=True, recurrent_dropout=0.2)(batch_1)
    dropout_1 = keras.layers.Dropout(0.5)(gru_1)
    batch_2 = keras.layers.BatchNormalization()(dropout_1)
    gru_out = keras.layers.GRU(64, recurrent_dropout=0.2)(batch_2)
    
    concat = keras.layers.concatenate([gru_out, inputB])
    dense_1 = keras.layers.Dense(60)(concat)
    dropout_2 = keras.layers.Dropout(0.5)(dense_1)
    concat_output = keras.layers.Dense(30)(dropout_2)
    output = keras.layers.Dense(output_dim)(concat_output)
    model = keras.Model(inputs=[inputA, inputB], outputs=[output], name=model_name)
    return model

> ?? **WHY** adding drop out layer and recurrent dropout will slow the training down so much? (or is it just because of Colab internet?)

> **ANSWER**: this seems to be the problem contributing from `recurrent_dropout`, which prevents Colab to use `cuDNN` kernels to speed up training

# CNN

In [ ]:
def create_model_3(inputA_feature_dim,
                   inputB_feature_dim,
                   output_dim,
                   mat_length,
                   model_name='KDI-s_KDI-xs_Dense-32-2'):
    inputA = keras.layers.Input(shape=[mat_length, mat_length, inputA_feature_dim], name='inputA')
    inputB = keras.layers.Input(shape=[mat_length, mat_length, inputB_feature_dim], name='inputB')
    
    user = keras.layers.Conv2D(32, (3, 3), 2, activation='relu')(inputA)
    user = keras.layers.Conv2D(32, (3, 3), activation='relu')(user)
    user = keras.layers.MaxPooling2D((2, 2))(user)
    user = keras.layers.Conv2D(64, (3, 3), activation='relu')(user)
    user = keras.layers.Conv2D(64, (3, 3), activation='relu')(user)
    user = keras.layers.MaxPooling2D((2, 2))(user)
    user = keras.layers.Flatten()(user)
    user = keras.layers.Dense(64, activation='relu')(user)
    
    keycode = keras.layers.Conv2D(16, (3, 3), 2, activation='relu')(inputB)
    keycode = keras.layers.Conv2D(16, (3, 3), activation='relu')(keycode)
    keycode = keras.layers.MaxPooling2D((2, 2))(keycode)
    keycode = keras.layers.Conv2D(32, (3, 3), activation='relu')(keycode)
    keycode = keras.layers.Conv2D(32, (3, 3), activation='relu')(keycode)
    keycode = keras.layers.MaxPooling2D((2, 2))(keycode)
    keycode = keras.layers.Flatten()(keycode)
    keycode = keras.layers.Dense(32, activation='relu')(keycode)

    concat = keras.layers.concatenate([user, keycode])
    concat = keras.layers.Dense(32, activity_regularizer=tf.keras.regularizers.L2(0.001))(concat)
    output = keras.layers.Dense(output_dim, activation='relu')(concat)

    model = keras.Model(inputs=[inputA, inputB], outputs=[output])
    return model

In [ ]:
def create_model_4(inputA_feature_dim,
                   inputB_feature_dim,
                   output_dim,
                   mat_length, 
                   model_name='KDI_LeNet5_Dense-64-2'):
    inputA = keras.layers.Input(shape=[mat_length, mat_length, inputA_feature_dim], name='inputA')
    inputB = keras.layers.Input(shape=[mat_length, mat_length, inputB_feature_dim], name='inputB')
    
    user = keras.layers.Conv2D(32, (3, 3), 2, activation='relu')(inputA)
    user = keras.layers.Conv2D(32, (3, 3), activation='relu')(user)
    user = keras.layers.AveragePooling2D()(user)
    user = keras.layers.Conv2D(64, (3, 3), activation='relu')(user)
    user = keras.layers.Conv2D(64, (3, 3), activation='relu')(user)
    user = keras.layers.Flatten()(user)
    user = keras.layers.Dense(64)(user)

    keycode = keras.layers.Conv2D(32, (3, 3), activation='relu')(inputB)
    keycode = keras.layers.AveragePooling2D()(keycode)
    keycode = keras.layers.Conv2D(32, (3, 3), activation='relu')(keycode)
    keycode = keras.layers.AveragePooling2D()(keycode)
    keycode = keras.layers.Conv2D(64, (3, 3), activation='relu')(keycode)
    keycode = keras.layers.AveragePooling2D()(keycode)
    keycode = keras.layers.Conv2D(64, (3, 3), activation='relu')(keycode)
    keycode = keras.layers.Flatten()(keycode)
    keycode = keras.layers.Dense(64)(keycode)

    concat = keras.layers.concatenate([user, keycode])
    concat = keras.layers.Dense(64, activity_regularizer=tf.keras.regularizers.L2(0.001))(concat)
    output = keras.layers.Dense(output_dim, activation='relu')(concat)

    model = keras.Model(inputs=[inputA, inputB], outputs=[output])
    return model

In [ ]:
def create_model_5(inputA_feature_dim,
                   inputB_feature_dim,
                   output_dim,
                   mat_length,
                   l1_reg,
                   model_name='KDI-s_KDI-xs_Dense-32-2-Reg'):
    inputA = keras.layers.Input(shape=[mat_length, mat_length, inputA_feature_dim], name='inputA')
    inputB = keras.layers.Input(shape=[mat_length, mat_length, inputB_feature_dim], name='inputB')
    
    user = keras.layers.Conv2D(32, (3, 3), 2, activation='relu')(inputA)
    user = keras.layers.Conv2D(32, (3, 3), activation='relu')(user)
    user = keras.layers.MaxPooling2D((2, 2))(user)
    user = keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=keras.regularizers.L1(l1=l1_reg))(user)
    user = keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=keras.regularizers.L1(l1=l1_reg))(user)
    user = keras.layers.MaxPooling2D((2, 2))(user)
    user = keras.layers.Flatten()(user)
    user = keras.layers.Dense(64, activation='relu')(user)
    
    keycode = keras.layers.Conv2D(16, (3, 3), 2, activation='relu')(inputB)
    keycode = keras.layers.Conv2D(16, (3, 3), activation='relu')(keycode)
    keycode = keras.layers.MaxPooling2D((2, 2))(keycode)
    keycode = keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=keras.regularizers.L1(l1=l1_reg))(keycode)
    keycode = keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=keras.regularizers.L1(l1=l1_reg))(keycode)
    keycode = keras.layers.MaxPooling2D((2, 2))(keycode)
    keycode = keras.layers.Flatten()(keycode)
    keycode = keras.layers.Dense(32, activation='relu')(keycode)

    concat = keras.layers.concatenate([user, keycode])
    concat = keras.layers.Dense(32, activity_regularizer=tf.keras.regularizers.L1(l1_reg))(concat)
    output = keras.layers.Dense(output_dim, activation='relu')(concat)

    model = keras.Model(inputs=[inputA, inputB], outputs=[output])
    return model

In [ ]:
def create_model_6(inputA_feature_dim,
                   inputB_feature_dim,
                   output_dim,
                   mat_length, 
                   l1_reg,
                   model_name='KDI_LeNet5_Dense-64-2-Reg'):
    inputA = keras.layers.Input(shape=[mat_length, mat_length, inputA_feature_dim], name='inputA')
    inputB = keras.layers.Input(shape=[mat_length, mat_length, inputB_feature_dim], name='inputB')
    
    user = keras.layers.Conv2D(32, (3, 3), 2, activation='relu')(inputA)
    user = keras.layers.Conv2D(32, (3, 3), activation='relu')(user)
    user = keras.layers.AveragePooling2D()(user)
    user = keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=keras.regularizers.L1(l1=l1_reg))(user)
    user = keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=keras.regularizers.L1(l1=l1_reg))(user)
    user = keras.layers.Flatten()(user)
    user = keras.layers.Dense(64)(user)

    keycode = keras.layers.Conv2D(32, (3, 3), activation='relu')(inputB)
    keycode = keras.layers.AveragePooling2D()(keycode)
    keycode = keras.layers.Conv2D(32, (3, 3), activation='relu')(keycode)
    keycode = keras.layers.AveragePooling2D()(keycode)
    keycode = keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=keras.regularizers.L1(l1=l1_reg))(keycode)
    keycode = keras.layers.AveragePooling2D()(keycode)
    keycode = keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=keras.regularizers.L1(l1=l1_reg))(keycode)
    keycode = keras.layers.Flatten()(keycode)
    keycode = keras.layers.Dense(64)(keycode)

    concat = keras.layers.concatenate([user, keycode])
    concat = keras.layers.Dense(64, activity_regularizer=tf.keras.regularizers.L1(l1=l1_reg))(concat)
    output = keras.layers.Dense(output_dim, activation='relu')(concat)

    model = keras.Model(inputs=[inputA, inputB], outputs=[output])
    return model

In [ ]:
def create_model_7(inputA_feature_dim,
                   inputB_feature_dim,
                   output_dim,
                   mat_length, 
                   l1_reg,
                   model_name='CNN_RNN_reg'):
    inputA = keras.layers.Input(shape=[mat_length, mat_length, inputA_feature_dim], name='inputA')
    inputB = keras.layers.Input(shape=[mat_length, mat_length, inputB_feature_dim], name='inputB')
    
    user = keras.layers.Conv2D(32, (3, 3), activation='relu')(inputA)
    user = keras.layers.MaxPooling2D()(user)
    user = keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=keras.regularizers.L1(l1=l1_reg))(user)
    user = keras.layers.AveragePooling2D()(user)
    user = keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=keras.regularizers.L1(l1=l1_reg))(user)
    user = keras.layers.Reshape((-1, 64))(user)

    keycode = keras.layers.Conv2D(32, (3, 3), activation='relu')(inputB)
    keycode = keras.layers.MaxPooling2D()(keycode)
    keycode = keras.layers.Conv2D(32, (3, 3), activation='relu')(keycode)
    keycode = keras.layers.MaxPooling2D()(keycode)
    keycode = keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=keras.regularizers.L1(l1=l1_reg))(keycode)
    keycode = keras.layers.Reshape((-1, 64))(keycode)

    concat = keras.layers.concatenate([user, keycode])
    concat = keras.layers.GRU(128, activation='relu')(concat)
    concat = keras.layers.Dense(64, activation='relu', activity_regularizer=tf.keras.regularizers.L1(l1=l1_reg))(concat)
    output = keras.layers.Dense(output_dim, activation='relu')(concat)

    model = keras.Model(inputs=[inputA, inputB], outputs=[output])
    return model

In [7]:
def create_model_8(inputA_feature_dim,
                   inputB_feature_dim,
                   output_dim,
                   mat_length,
                   model_name='KDI-s_KDI-xs_Dense-nonReLU'):
    inputA = keras.layers.Input(shape=[mat_length, mat_length, inputA_feature_dim], name='inputA')
    inputB = keras.layers.Input(shape=[mat_length, mat_length, inputB_feature_dim], name='inputB')
    
    user = keras.layers.Conv2D(32, (3, 3), 2)(inputA)
    user = keras.layers.Conv2D(32, (3, 3))(user)
    user = keras.layers.MaxPooling2D((2, 2))(user)
    user = keras.layers.Conv2D(64, (3, 3))(user)
    user = keras.layers.Conv2D(64, (3, 3))(user)
    user = keras.layers.MaxPooling2D((2, 2))(user)
    user = keras.layers.Flatten()(user)
    # user = keras.layers.Dense(64, activation='relu')(user)
    
    keycode = keras.layers.Conv2D(16, (3, 3), 2)(inputB)
    keycode = keras.layers.Conv2D(16, (3, 3))(keycode)
    keycode = keras.layers.MaxPooling2D((2, 2))(keycode)
    keycode = keras.layers.Conv2D(32, (3, 3))(keycode)
    keycode = keras.layers.Conv2D(32, (3, 3))(keycode)
    keycode = keras.layers.MaxPooling2D((2, 2))(keycode)
    keycode = keras.layers.Flatten()(keycode)
    # keycode = keras.layers.Dense(32, activation='relu')(keycode)

    concat = keras.layers.concatenate([user, keycode])
    concat = keras.layers.Dense(128, activity_regularizer=tf.keras.regularizers.L1(0.001))(concat)
    concat = keras.layers.Dense(32)(concat)
    output = keras.layers.Dense(output_dim)(concat)

    model = keras.Model(inputs=[inputA, inputB], outputs=[output])
    return model


In [12]:
def create_model_9(inputA_feature_dim,
                   inputB_feature_dim,
                   output_dim,
                   mat_length,
                   model_name='KDI-s_KDI-xs_Dense-ReLU'):
    inputA = keras.layers.Input(shape=[mat_length, mat_length, inputA_feature_dim], name='inputA')
    inputB = keras.layers.Input(shape=[mat_length, mat_length, inputB_feature_dim], name='inputB')
    
    user = keras.layers.Conv2D(32, (3, 3), 2, activation='relu')(inputA)
    user = keras.layers.Conv2D(32, (3, 3), activation='relu')(user)
    user = keras.layers.MaxPooling2D((2, 2))(user)
    user = keras.layers.Conv2D(64, (3, 3), activation='relu')(user)
    user = keras.layers.Conv2D(64, (3, 3), activation='relu')(user)
    user = keras.layers.MaxPooling2D((2, 2))(user)
    user = keras.layers.Flatten()(user)
    user = keras.layers.Dense(128, activation='relu')(user)
    
    keycode = keras.layers.Conv2D(16, (3, 3), 2, activation='relu')(inputB)
    keycode = keras.layers.Conv2D(16, (3, 3), activation='relu')(keycode)
    keycode = keras.layers.MaxPooling2D((2, 2))(keycode)
    keycode = keras.layers.Conv2D(32, (3, 3), activation='relu')(keycode)
    keycode = keras.layers.Conv2D(32, (3, 3), activation='relu')(keycode)
    keycode = keras.layers.MaxPooling2D((2, 2))(keycode)
    keycode = keras.layers.Flatten()(keycode)
    keycode = keras.layers.Dense(64, activation='relu')(keycode)

    concat = keras.layers.concatenate([user, keycode])
    concat = keras.layers.Dense(128, activity_regularizer=tf.keras.regularizers.L1(0.001), activation='relu')(concat)
    concat = keras.layers.Dense(32)(concat)
    output = keras.layers.Dense(output_dim)(concat)

    model = keras.Model(inputs=[inputA, inputB], outputs=[output])
    return model

In [14]:
def create_model_10(inputA_feature_dim,
                   inputB_feature_dim,
                   output_dim,
                   mat_length,
                   model_name='KDI-s_KDI-xs_Dense-ReLU-l2reg'):
    inputA = keras.layers.Input(shape=[mat_length, mat_length, inputA_feature_dim], name='inputA')
    inputB = keras.layers.Input(shape=[mat_length, mat_length, inputB_feature_dim], name='inputB')
    
    user = keras.layers.Conv2D(32, (3, 3), 2, activation='relu')(inputA)
    user = keras.layers.Conv2D(32, (3, 3), activation='relu')(user)
    user = keras.layers.MaxPooling2D((2, 2))(user)
    user = keras.layers.Conv2D(64, (3, 3), activation='relu')(user)
    user = keras.layers.Conv2D(64, (3, 3), activation='relu')(user)
    user = keras.layers.MaxPooling2D((2, 2))(user)
    user = keras.layers.Flatten()(user)
    user = keras.layers.Dense(128, activation='relu')(user)
    
    keycode = keras.layers.Conv2D(16, (3, 3), 2, activation='relu')(inputB)
    keycode = keras.layers.Conv2D(16, (3, 3), activation='relu')(keycode)
    keycode = keras.layers.MaxPooling2D((2, 2))(keycode)
    keycode = keras.layers.Conv2D(32, (3, 3), activation='relu')(keycode)
    keycode = keras.layers.Conv2D(32, (3, 3), activation='relu')(keycode)
    keycode = keras.layers.MaxPooling2D((2, 2))(keycode)
    keycode = keras.layers.Flatten()(keycode)
    keycode = keras.layers.Dense(64, activation='relu')(keycode)

    concat = keras.layers.concatenate([user, keycode])
    concat = keras.layers.Dense(128, activity_regularizer=tf.keras.regularizers.L2(0.0001), activation='relu')(concat)
    concat = keras.layers.Dense(64, activation='relu')(concat)
    concat = keras.layers.Dense(32)(concat)
    output = keras.layers.Dense(output_dim)(concat)

    model = keras.Model(inputs=[inputA, inputB], outputs=[output])
    return model

In [ ]:
def create_model_11(inputA_feature_dim,
                   inputB_feature_dim,
                   output_dim,
                   mat_length,
                   model_name='KDI-s_KDI-xs_Dense-ReLU-l1l2reg'):
    inputA = keras.layers.Input(shape=[mat_length, mat_length, inputA_feature_dim], name='inputA')
    inputB = keras.layers.Input(shape=[mat_length, mat_length, inputB_feature_dim], name='inputB')
    
    user = keras.layers.Conv2D(32, (3, 3), 2, activation='relu')(inputA)
    user = keras.layers.Conv2D(32, (3, 3), activation='relu')(user)
    user = keras.layers.MaxPooling2D((2, 2))(user)
    user = keras.layers.Conv2D(64, (3, 3), activation='relu')(user)
    user = keras.layers.Conv2D(64, (3, 3), activation='relu')(user)
    user = keras.layers.MaxPooling2D((2, 2))(user)
    user = keras.layers.Flatten()(user)
    user = keras.layers.Dense(128, activation='relu')(user)
    
    keycode = keras.layers.Conv2D(16, (3, 3), 2, activation='relu')(inputB)
    keycode = keras.layers.Conv2D(16, (3, 3), activation='relu')(keycode)
    keycode = keras.layers.MaxPooling2D((2, 2))(keycode)
    keycode = keras.layers.Conv2D(32, (3, 3), activation='relu')(keycode)
    keycode = keras.layers.Conv2D(32, (3, 3), activation='relu')(keycode)
    keycode = keras.layers.MaxPooling2D((2, 2))(keycode)
    keycode = keras.layers.Flatten()(keycode)
    keycode = keras.layers.Dense(64, activation='relu')(keycode)

    concat = keras.layers.concatenate([user, keycode])
    concat = keras.layers.Dense(128, activity_regularizer=tf.keras.regularizers.L1L2(0.0001), activation='relu')(concat)
    concat = keras.layers.Dense(64, activation='relu')(concat)
    concat = keras.layers.Dense(32, activation='relu')(concat)
    output = keras.layers.Dense(output_dim, activation='relu')(concat)

    model = keras.Model(inputs=[inputA, inputB], outputs=[output])
    return model

In [30]:
def create_model_12(inputA_feature_dim,
                   inputB_feature_dim,
                   output_dim,
                   mat_length,
                   model_name='KDI-s_KDI-xs_Dense-ReLU-noreg'):
    inputA = keras.layers.Input(shape=[mat_length, mat_length, inputA_feature_dim], name='inputA')
    inputB = keras.layers.Input(shape=[mat_length, mat_length, inputB_feature_dim], name='inputB')
    
    user = keras.layers.Conv2D(32, (3, 3), 2, activation='relu')(inputA)
    user = keras.layers.Conv2D(32, (3, 3), activation='relu')(user)
    user = keras.layers.MaxPooling2D((2, 2))(user)
    user = keras.layers.Conv2D(64, (3, 3), activation='relu')(user)
    user = keras.layers.Conv2D(64, (3, 3), activation='relu')(user)
    user = keras.layers.MaxPooling2D((2, 2))(user)
    user = keras.layers.Flatten()(user)
    user = keras.layers.Dense(128, activation='relu')(user)
    user = keras.layers.Dense(64, activation='relu')(user)
    user = keras.layers.Dense(32, activation='relu')(user)
    
    keycode = keras.layers.Conv2D(16, (3, 3), 2, activation='relu')(inputB)
    keycode = keras.layers.Conv2D(16, (3, 3), activation='relu')(keycode)
    keycode = keras.layers.MaxPooling2D((2, 2))(keycode)
    keycode = keras.layers.Conv2D(32, (3, 3), activation='relu')(keycode)
    keycode = keras.layers.Conv2D(64, (3, 3), activation='relu')(keycode)
    keycode = keras.layers.MaxPooling2D((2, 2))(keycode)
    keycode = keras.layers.Flatten()(keycode)
    keycode = keras.layers.Dense(128, activation='relu')(keycode)

    concat = keras.layers.concatenate([user, keycode])
    concat = keras.layers.Dense(80, activation='relu')(concat)
    concat = keras.layers.Dense(40, activation='relu')(concat)
    concat = keras.layers.Dense(20, activation='relu')(concat)
    output = keras.layers.Dense(output_dim, activation='relu')(concat)

    model = keras.Model(inputs=[inputA, inputB], outputs=[output])
    return model

In [8]:
def create_model_13(inputA_feature_dim,
                   inputB_feature_dim,
                   output_dim,
                   mat_length,
                   model_name='KDI-1s_KDI-xs_GRU'):
    inputA = keras.layers.Input(shape=[mat_length, mat_length, inputA_feature_dim], name='inputA')
    inputB = keras.layers.Input(shape=[mat_length, mat_length, inputB_feature_dim], name='inputB')
    
    user = keras.layers.Conv2D(32, (3, 3), 2, activation='relu')(inputA)
    user = keras.layers.MaxPooling2D((2, 2))(user)
    user = keras.layers.Conv2D(64, (3, 3), activation='relu')(user)
    user = keras.layers.MaxPooling2D((2, 2))(user)
    user = keras.layers.Conv2D(128, (3, 3), activation='relu')(user)
    user = keras.layers.Flatten()(user)
    user = keras.layers.Dense(128, activation='relu')(user)
    user = keras.layers.Dense(32, activation='relu')(user)
    user = keras.layers.Dense(8, activation='relu')(user)
    user = tf.repeat(tf.expand_dims(user, axis=1), 49, axis=1)
    
    keycode = keras.layers.Conv2D(32, (3, 3), activation='relu')(inputB)
    keycode = keras.layers.Conv2D(32, (3, 3), activation='relu')(keycode)
    keycode = keras.layers.MaxPooling2D((2, 2))(keycode)
    keycode = keras.layers.Conv2D(64, (3, 3), activation='relu')(keycode)
    keycode = keras.layers.Conv2D(64, (3, 3), activation='relu')(keycode)
    keycode = keras.layers.MaxPooling2D((2, 2))(keycode)
    keycode = keras.layers.Reshape((-1, 64))(keycode)

    concat = keras.layers.concatenate([user, keycode])
    concat = keras.layers.GRU(72, return_sequences=True)(concat)
    concat = keras.layers.GRU(72)(concat)
    concat = keras.layers.Dense(32, activation='relu')(concat)
    output = keras.layers.Dense(output_dim, activation='relu')(concat)

    model = keras.Model(inputs=[inputA, inputB], outputs=[output])
    return model

In [ ]:
# create_model_13(inputA_feature_dim=71,
#                    inputB_feature_dim=71,
#                    output_dim=1,
#                    mat_length=40).summary()

# KeycodeToTime

In [72]:
def create_model_keyTOtime(features_dim,
                           output_dim,
                           model_name='Keycode_TO_Time'):
    model = keras.models.Sequential([
        keras.layers.GRU(128, return_sequences=True, input_shape=[None, features_dim], dropout=0.2, recurrent_dropout=0.2),
        keras.layers.GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
        keras.layers.TimeDistributed(keras.layers.Dense(output_dim))
    ])
    return model

In [ ]:
def create_model_keyTotime_CNN(features_dim, 
                               output_dim, 
                               window_length,
                               model_name='Keycode_TO_Time_CNN'):
    model = keras.models.Sequential([
            keras.layers.Conv2D(32, (2, features_dim), input_shape=[window_length, features_dim, 1], activation='relu'),
            keras.layers.Dense(output_dim, activation='relu', kernel_regularizer=tf.keras.regularizers.L1(0.001))])
    return model

In [ ]:
def keycodeTotime_prep(uni_df, latency, window_length, shift, batch_size):
    df = np.concatenate([KEYCODE_enc.transform(uni_df[['KEYCODE']]).toarray(), uni_df[['INDEX', latency]]], axis=1)
    ds = tf.data.Dataset.from_tensor_slices(df).window(size=window_length, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(window_length)).batch(batch_size)
    ds = ds.map(lambda windows: (windows[:, :, :-1], windows[:, :, -1]))
    return ds

In [ ]:
def keycodeTotime_prep_CNN(uni_df, latency, window_length, shift, batch_size):
    df = np.concatenate([KEYCODE_enc.transform(uni_df[['KEYCODE']]).toarray(), uni_df[['INDEX', latency]]], axis=1)
    ds = tf.data.Dataset.from_tensor_slices(df).window(size=window_length, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(window_length)).batch(batch_size)
    ds = ds.map(lambda windows: (windows[:, :, :-1], windows[:, :, -1]))
    ds = ds.map(lambda input, output: (tf.expand_dims(input, axis=-1), output))
    return ds

In [ ]:
def structure_one(preprocessed_data, encoder, window_length, shift, batch_size):
    '''
    Keycode to time (only one kind of time latency, i.e. output_dim=1)
    '''
    df = preprocessed_data.drop(columns=['USER'])
    df = np.concatenate([encoder.transform(df[['KEYCODE']].astype(str)).toarray(), df.drop(columns=['KEYCODE'])], axis=1)
    dataset = tf.data.Dataset.from_tensor_slices(df).window(size=window_length, shift=shift, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length)).batch(batch_size)
    dataset = dataset.map(lambda windows: (windows[:, :, :-1], windows[:, :, -1]))
    return dataset

## XX_key_df : unigraph but with only HL, i.e. ['USER', 'KEYCODE', 'INDEX', 'HL']
# train_key_ds = structure_one(train_key_df, encoder_uni, window_length=31, shift=1, batch_size=32)
# test_key_ds = structure_one(test_key_df, encoder_uni, window_length=31, shift=1, batch_size=32)
# train_key_ds_avg = structure_one(train_key_df_avg, encoder_uni, window_length=31, shift=1, batch_size=32)
# test_key_ds_avg = structure_one(test_key_df_avg, encoder_uni, window_length=31, shift=1, batch_size=32)

def ONE_base(name='KeyToCode-HL'):
    model_ONE_base = keras.models.Sequential([
        keras.layers.GRU(128, return_sequences=True, input_shape=[None, input_dim], dropout=0.2, recurrent_dropout=0.2),
        keras.layers.GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
        keras.layers.TimeDistributed(keras.layers.Dense(1))
    ])
    return model_ONE_base

In [ ]:
def structure_one_pl(preprocessed_data, encoder, window_length, shift, batch_size):
    df = preprocessed_data.drop(columns=['USER'])
    df = np.concatenate([encoder.transform(df[['KEYCODE']].astype(str)).toarray(), df.drop(columns=['KEYCODE'])], axis=1)
    dataset = tf.data.Dataset.from_tensor_slices(df).window(size=window_length, shift=shift, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length)).batch(batch_size)
    dataset = dataset.map(lambda windows: (windows[:, :, :-2], windows[:, :, -2:]))
    return dataset

## XX_df_uni : unigraph,  i.e. ['USER', 'KEYCODE', 'INDEX', 'HL', 'PL]
# train_key_pl_ds = structure_one_pl(train_df_uni, encoder_uni, window_length=31, shift=1, batch_size=32)
# test_key_pl_ds = structure_one_pl(test_df_uni, encoder_uni, window_length=31, shift=1, batch_size=32)
# train_key_pl_ds_avg = structure_one_pl(train_df_uni_avg, encoder_uni, window_length=31, shift=1, batch_size=32)
# test_key_pl_ds_avg = structure_one_pl(test_df_uni_avg, encoder_uni, window_length=31, shift=1, batch_size=32)

def ONE_base_pl_avg(name='KeyToCode-HLPL'):
    model_ONE_base_pl_avg = keras.models.Sequential([
        keras.layers.GRU(128, return_sequences=True, input_shape=[None, input_dim], dropout=0.2, recurrent_dropout=0.2),
        keras.layers.GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
        keras.layers.TimeDistributed(keras.layers.Dense(2))
    ])
    return model_ONE_base_pl_avg

# Old Models

In [ ]:
def model_ONE_base(name='ONE_base'):
    model_ONE_base = keras.models.Sequential([
        keras.layers.GRU(128, return_sequences=True, input_shape=[None, unit_time_depth], dropout=0.2, recurrent_dropout=0.2),
        keras.layers.GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
        keras.layers.TimeDistributed(keras.layers.Dense(1))
    ])
    return model_ONE_Base

def model_ONE_1(name='ONE_1_mae'):
    model_ONE_1 = keras.models.Sequential([
        keras.layers.GRU(128, return_sequences=True, input_shape=[None, unit_time_depth], dropout=0.2, recurrent_dropout=0.2),
        keras.layers.GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
        keras.layers.TimeDistributed(keras.layers.Dense(1, activity_regularizer=tf.keras.regularizers.L2(0.01)))
    ])
    return model_ONE_1

def model_ONE_2(name=''):
    model_ONE_2 = keras.models.Sequential([
        keras.layers.GRU(128, return_sequences=True, input_shape=[None, unit_time_depth], dropout=0.2, recurrent_dropout=0.2),
        keras.layers.GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
        keras.layers.TimeDistributed(keras.layers.Dense(1, activity_regularizer=tf.keras.regularizers.L2(0.001)))
    ])
    return model_ONE_2

def model_ONE_3(name=''):
    model_ONE_3 = keras.models.Sequential([
        keras.layers.GRU(128, return_sequences=True, input_shape=[None, unit_time_depth], dropout=0.2, recurrent_dropout=0.2),
        keras.layers.GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
        keras.layers.TimeDistributed(keras.layers.Dense(32, activity_regularizer=tf.keras.regularizers.L2(0.001))),
        keras.layers.TimeDistributed(keras.layers.Dense(1))
    ])
    return model_ONE_3

def model_ONE_4(name=''):
    model_ONE_4 = keras.models.Sequential([
        keras.layers.GRU(128, return_sequences=True, input_shape=[None, unit_time_depth], dropout=0.2, recurrent_dropout=0.2),
        keras.layers.GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
        keras.layers.TimeDistributed(keras.layers.Dense(64, activity_regularizer=tf.keras.regularizers.L2(0.01))),
        keras.layers.TimeDistributed(keras.layers.Dense(32, activity_regularizer=tf.keras.regularizers.L2(0.01))),
        keras.layers.TimeDistributed(keras.layers.Dense(1))
    ])
    return model_ONE_4

In [ ]:
def CNN_base_mae(mat_length, 
                 unit_time_depth,
                 name='CNN_base_mae'):
    input_1 = keras.layers.Input(shape=[mat_length, mat_length, 6], name='input_kdi')
    input_2 = keras.layers.Input(shape=[unit_time_depth], name='keycode')
    conv2d_1 = keras.layers.Conv2D(64, (3, 3))(input_1)
    conv2d_2 = keras.layers.Conv2D(64, (3, 3))(conv2d_1)
    maxpool_1 = keras.layers.MaxPooling2D((2, 2))(conv2d_2)
    conv2d_3 = keras.layers.Conv2D(128, (3, 3))(maxpool_1)
    conv2d_4 = keras.layers.Conv2D(128, (3, 3))(conv2d_3)
    maxpool_2 = keras.layers.MaxPooling2D((2, 2))(conv2d_4)
    reshape_1 = keras.layers.Reshape((-1, 128))(maxpool_2)
    gru_1 = keras.layers.GRU(128, recurrent_dropout=0.2)(reshape_1)
    dense_1 = keras.layers.Dense(64, activity_regularizer=tf.keras.regularizers.L2(0.001))(gru_1)
    concat = keras.layers.concatenate([dense_1, input_2])
    dense_3 = keras.layers.Dense(64, activity_regularizer=tf.keras.regularizers.L2(0.001))(concat)
    output = keras.layers.Dense(2)(dense_3)
    model_base = keras.Model(inputs=[input_1, input_2], outputs=[output])
    return model_base

In [ ]:
def model_Exp2Gp0(train_kds_uni_mean, model_name = 'Exp2Gp0'):
    feature_dim = train_kds_uni_mean.inputA[-1]
    output_dim = train_kds_uni_mean.output[-1]
    user_gru = 64
    concat_gru = 32

    inputA = keras.layers.Input(shape=[None, feature_dim], name='InputA')
    inputB = keras.layers.Input(shape=[feature_dim-output_dim], name='InputB')

    batch_1 = keras.layers.BatchNormalization()(inputA)
    gru_1 = keras.layers.GRU(user_gru, return_sequences=True)(batch_1)
    dropout_1 = keras.layers.Dropout(0.5)(gru_1)
    batch_2 = keras.layers.BatchNormalization()(dropout_1)
    gru_out = keras.layers.GRU(user_gru)(batch_2)

    concat = keras.layers.concatenate([gru_out, inputB])

    reshape = keras.layers.Reshape((user_gru+feature_dim-output_dim, 1))(concat)
    concat_output = keras.layers.GRU(concat_gru = 32)(reshape)

    output = keras.layers.Dense(output_dim)(concat_output)
    model_Exp2Gp0 = keras.Model(inputs=[inputA, inputB], outputs=[output], name=model_name)
    return model_Exp2Gp0

def model_NewExp0Gp0(model_name = 'NewExp0Gp0'):
    feature_dim = train_kds_uni_mean.inputA[-1]
    output_dim = train_kds_uni_mean.output[-1]
    user_gru = 128
    concat_gru = 64

    inputA = keras.layers.Input(shape=[None, feature_dim], name='InputA')
    inputB = keras.layers.Input(shape=[feature_dim-output_dim], name='InputB')

    batch_1 = keras.layers.BatchNormalization()(inputA)
    gru_1 = keras.layers.GRU(user_gru, return_sequences=True)(batch_1)
    dropout_1 = keras.layers.Dropout(0.5)(gru_1)
    batch_2 = keras.layers.BatchNormalization()(dropout_1)
    gru_out = keras.layers.GRU(user_gru)(batch_2)

    concat = keras.layers.concatenate([gru_out, inputB])

    reshape = keras.layers.Reshape((user_gru+feature_dim-output_dim, 1))(concat)
    concat_output = keras.layers.GRU(concat_gru)(reshape)

    output = keras.layers.Dense(output_dim)(concat_output)
    model_NewExp0Gp0 = keras.Model(inputs=[inputA, inputB], outputs=[output], name=model_name)
    return model_NewExp0Gp0


def model_NewExp0Gp1(model_name='NewExp0Gp1')
    feature_dim = train_kds_uni_mean.inputA[-1]
    output_dim = train_kds_uni_mean.output[-1]
    user_gru = 64
    # concat_gru = 32

    inputA = keras.layers.Input(shape=[None, feature_dim], name='InputA')
    inputB = keras.layers.Input(shape=[feature_dim-output_dim], name='InputB')

    batch_1 = keras.layers.BatchNormalization()(inputA)
    gru_1 = keras.layers.GRU(user_gru, return_sequences=True)(batch_1)
    dropout_1 = keras.layers.Dropout(0.5)(gru_1)
    batch_2 = keras.layers.BatchNormalization()(dropout_1)
    gru_out = keras.layers.GRU(user_gru)(batch_2)

    concat = keras.layers.concatenate([gru_out, inputB])

    # reshape = keras.layers.Reshape((user_gru+feature_dim-output_dim, 1))(concat)
    # concat_output = keras.layers.GRU(concat_gru)(reshape)

    dense_1 = keras.layers.Dense(60)(concat)
    concat_output = keras.layers.Dense(30)(dense_1)

    output = keras.layers.Dense(output_dim)(concat_output)
    model_NewExp0Gp1 = keras.Model(inputs=[inputA, inputB], outputs=[output], name=model_name)
    return model_NewExp0Gp1


def model_NewExp0Gp2(model_name='NewExp0Gp2'):
    feature_dim = train_kds.inputA[-1]
    output_dim = train_kds.output[-1]
    user_gru = 64
    # concat_gru = 32

    inputA = keras.layers.Input(shape=[None, feature_dim], name='InputA')
    inputB = keras.layers.Input(shape=[feature_dim-output_dim], name='InputB')

    batch_1 = keras.layers.BatchNormalization()(inputA)
    gru_1 = keras.layers.GRU(user_gru, return_sequences=True)(batch_1)
    dropout_1 = keras.layers.Dropout(0.5)(gru_1)
    batch_2 = keras.layers.BatchNormalization()(dropout_1)
    gru_out = keras.layers.GRU(user_gru)(batch_2)

    concat = keras.layers.concatenate([gru_out, inputB])

    # reshape = keras.layers.Reshape((user_gru+feature_dim-output_dim, 1))(concat)
    # concat_output = keras.layers.GRU(concat_gru)(reshape)

    dense_1 = keras.layers.Dense(60)(concat)
    concat_output = keras.layers.Dense(30)(dense_1)

    output = keras.layers.Dense(output_dim)(concat_output)
    model_NewExp0Gp2 = keras.Model(inputs=[inputA, inputB], outputs=[output], name=model_name)
    return model_NewExp0Gp2


def model_NewExp0Gp3(model_name='NewExp0Gp3'):
    feature_dim = train_kds.inputA[-1]
    output_dim = train_kds.output[-1]
    user_gru = 64
    # concat_gru = 32

    inputA = keras.layers.Input(shape=[None, feature_dim], name='InputA')
    inputB = keras.layers.Input(shape=[feature_dim-output_dim], name='InputB')

    batch_1 = keras.layers.BatchNormalization()(inputA)
    gru_1 = keras.layers.GRU(user_gru, return_sequences=True)(batch_1)
    dropout_1 = keras.layers.Dropout(0.5)(gru_1)
    batch_2 = keras.layers.BatchNormalization()(dropout_1)
    gru_out = keras.layers.GRU(user_gru)(batch_2)

    concat = keras.layers.concatenate([gru_out, inputB])

    # reshape = keras.layers.Reshape((user_gru+feature_dim-output_dim, 1))(concat)
    # concat_output = keras.layers.GRU(concat_gru)(reshape)

    dense_1 = keras.layers.Dense(60)(concat)
    concat_output = keras.layers.Dense(30)(dense_1)

    output = keras.layers.Dense(output_dim)(concat_output)
    model_NewExp0Gp3 = keras.Model(inputs=[inputA, inputB], outputs=[output], name=model_name)
    return model_NewExp0Gp3


def model_NExp1Gp0(model_name='NExp1Gp0')
    feature_dim = train_kds_uni_mean.inputA[-1]
    output_dim = train_kds_uni_mean.output[-1]
    user_gru = 64
    concat_gru = 32

    inputA = keras.layers.Input(shape=[None, feature_dim], name='InputA')
    inputB = keras.layers.Input(shape=[feature_dim-output_dim], name='InputB')

    batch_1 = keras.layers.BatchNormalization()(inputA)
    gru_1 = keras.layers.GRU(user_gru, return_sequences=True)(batch_1)
    dropout_1 = keras.layers.Dropout(0.5)(gru_1)
    batch_2 = keras.layers.BatchNormalization()(dropout_1)
    gru_out = keras.layers.GRU(user_gru)(batch_2)

    concat = keras.layers.concatenate([gru_out, inputB])

    reshape = keras.layers.Reshape((user_gru+feature_dim-output_dim, 1))(concat)
    concat_output = keras.layers.GRU(concat_gru)(reshape)

    output = keras.layers.Dense(output_dim)(concat_output)
    model_NExp1Gp0 = keras.Model(inputs=[inputA, inputB], outputs=[output], name=model_name)
    return model_NExp1Gp0

## TypeNet-base_no-keycode_ConcatRNN-base

In [ ]:
## base model of Exp1Gp0

def typenet_base(inputA):
    name = 'TypeNet-base'
    batch_1 = keras.layers.BatchNormalization()(inputA)
    lstm_1 = keras.layers.LSTM(128, return_sequences=True)(batch_1)
    dropout_1 = keras.layers.Dropout(0.5)(lstm_1)
    batch_2 = keras.layers.BatchNormalization()(dropout_1)
    lstm_2 = keras.layers.LSTM(128)(batch_2)
    return name, lstm_2

def concate_RNN_base(concat, feature_dim, output_dim):
    name = 'ConcatRNN-base'
    reshape = keras.layers.Reshape((128+feature_dim-output_dim, 1))(concat)
    gru_1 = keras.layers.GRU(64)(reshape)
    return name, gru_1

def create_model(feature_dim, output_dim, user_embedding=None, keycode_embedding=None, concat_model=None):
    inputA = keras.layers.Input(shape=[None, feature_dim], name='InputA')
    inputB = keras.layers.Input(shape=[feature_dim-output_dim], name='InputB')
    
    if user_embedding:
        user_name, user_embeded = user_embedding(inputA)
    else:
        user_name, user_embeded = 'no-user', inputA
    if keycode_embedding:
        keycode_name, keycode_embeded = keycode_embedding(inputB)
    else:
        keycode_name, keycode_embeded = 'no-keycode', inputB
    concat = keras.layers.concatenate([user_embeded, keycode_embeded])

    if concat_model:
        concat_name, concat_output = concat_model(concat, feature_dim, output_dim)
    else:
        concat_name, concat_output = 'no-concat', concat
    output = keras.layers.Dense(output_dim)(concat_output)

    model_name = user_name + '_' + keycode_name + '_' + concat_name
    return keras.Model(inputs=[inputA, inputB], outputs=[output], name=model_name)

def train_model(feature_dim, output_dim, user_embedding, keycode_embedding, concat_model,
                optimizer, loss, metrics,
                trainset, testset, EPOCH, patience, avg_mode):
    model = create_model(feature_dim, output_dim, user_embedding, keycode_embedding, concat_model)
    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=metrics)
    history = model.fit(trainset, epochs=EPOCH, validation_data=testset, callbacks=get_callbacks(model.name, patience, avg_mode))
    return history, model

# IMPORT CHECK

In [ ]:
print("\n\nHello from Model_Architecture_List!")



Hello from Model_Architecture_List!
